# Tutorial on Using AutoGen with Local LLMs via FastChat

This tutorial aims to provide a detailed guide on setting up and using AutoGen with local Large Language Models (LLMs) using FastChat. This will enable you to harness the power of local LLMs for various applications without relying on external API services.

## Prerequisites

1. Python 3.8 or higher: Ensure you have Python installed.
2. AutoGen: Install AutoGen for managing LLM applications.
3. FastChat: Install FastChat for hosting and managing local LLMs.
4. LLM Model: A local LLM model such as Vicuna, available from sources like Hugging Face.

In [4]:
### Install Prerequisites
!pip3 install "fschat[model_worker,webui]"
!pip install python-dotenv huggingface-hub -q


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Create and store dotenv file

```
# Hugging Face API Token
HUGGINGFACE_TOKEN=your_huggingface_token_here

# Additional configuration variables can be added below
# MODEL_NAME=vicuna-7b-v1.5
# OUTPUT_DIR=./output
```


## Download and Prepare Your Model
Create a directory for your project and download your model checkpoint. For this tutorial, we'll use Vicuna-7B from Hugging Face.

In [3]:
import os
import subprocess
import shutil
from pathlib import Path
from dotenv import load_dotenv
from huggingface_hub import login

def run_command(command):
    try:
        result = subprocess.run(command, check=True, text=True, capture_output=True)
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"Error executing command: {e}")
        return None

def setup_llm_project(project_name="my_llm_project", model_repo="lmsys/vicuna-7b-v1.5"):
    project_path = Path(project_name)
    
    # Check if project folder exists
    if project_path.exists():
        print(f"Project folder '{project_name}' already exists.")
        return

    try:
        # Load environment variables
        load_dotenv()
        hf_token = os.getenv("HUGGINGFACE_TOKEN")
        if not hf_token:
            raise EnvironmentError("HUGGINGFACE_TOKEN not found in .env file")

        # Login to Hugging Face
        login(token=hf_token)
        print("Successfully logged in to Hugging Face")

        # Create and change to project directory
        project_path.mkdir(parents=True, exist_ok=True)
        os.chdir(project_path)
        print(f"Created and changed to directory: {project_path.resolve()}")

        # Check if git is installed
        if shutil.which("git") is None:
            raise EnvironmentError("Git is not installed or not in PATH.")

        # Clone the model repository
        clone_command = ["git", "clone", f"https://huggingface.co/{model_repo}"]
        output = run_command(clone_command)
        
        if output is not None:
            print(f"Successfully cloned {model_repo}")
        else:
            print(f"Failed to clone {model_repo}")

        # Additional setup steps can be added here
        
        print("LLM project setup completed successfully.")
    except Exception as e:
        print(f"An error occurred during setup: {e}")
        # Cleanup: remove the project directory if it was created
        if project_path.exists():
            shutil.rmtree(project_path)
        print("Setup failed. Project directory has been removed.")

setup_llm_project()

Created and changed to directory: /home/research/Asgharpoor/Agent/my_llm_project/my_llm_project
Successfully cloned lmsys/vicuna-7b-v1.5
LLM project setup completed successfully.


## Launch FastChat Components
Start the FastChat controller, model worker, and RESTful API server
You must run each command in a seprate terminal:

### Launch the controller:

```
python -m fastchat.serve.controller
```

### Launch the model worker:

```
python3 -m fastchat.serve.model_worker --model-path lmsys/vicuna-7b-v1.5
```

### Launch the API server:

```
python3 -m fastchat.serve.openai_api_server --host localhost --port 8000
```


## Configure AutoGen to Use the Local LLM

Modify your AutoGen configuration to connect to the local FastChat server. This is done by updating the 'OAI_CONFIG_LIST':



In [6]:
config_list = [
    {
        "model": "vicuna-7b-v1.5",
        "base_url": "http://localhost:8000/v1",
        "api_type": "openai",
        "api_key": "NULL"
    }
]


## Implement AutoGen with Local LLMs
Create a Python script (e.g., `autogen_local.py`) with the following code to set up and run AutoGen:



In [11]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

llm_config = {
    "config_list": [
        {
            "model": "vicuna-7b-v1.5",
            "base_url": "http://localhost:8000/v1",
            "api_type": "openai",
            "api_key": "NULL"
        }
    ]
}

philosopher = AssistantAgent(
    name="Philosopher",
    system_message="You provide profound insights and philosophical perspectives on the given topic.",
    llm_config=llm_config,
)

skeptic = AssistantAgent(
    name="Skeptic",
    system_message="You critically examine and question the arguments provided by the philosopher.",
    llm_config=llm_config,
)

moderator = UserProxyAgent(
    name="Moderator",
    system_message="A human admin who guides the discussion and poses philosophical questions.",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

groupchat = GroupChat(agents=[moderator, philosopher, skeptic], messages=[], max_round=12)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

chat_res = moderator.initiate_chat(manager, message="Discuss the concept of 'nothing'.", clear_history=False, config_list=llm_config["config_list"])


Moderator (to chat_manager):

Discuss the concept of 'nothing'.

--------------------------------------------------------------------------------
[autogen.oai.client: 08-06 10:25:34] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 08-06 10:25:42] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 08-06 10:25:45] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: Philosopher

[autogen.oai.client: 08-06 10:25:56] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"p

KeyboardInterrupt: 

## Additional Examples
### Example 1: Plotting Stock Prices
To plot stock prices using the same setup, modify the script to initiate the chat with a different request:

In [15]:

data_analyst = AssistantAgent(
    name="DataAnalyst",
    system_message="You analyze the given stock data and generate a plot showing the price changes.",
    llm_config=llm_config,
)

quality_assessor = AssistantAgent(
    name="QualityAssessor",
    system_message="You ensure the accuracy and clarity of the data analysis and the resulting plot.",
    llm_config=llm_config,
)

moderator = UserProxyAgent(
    name="Moderator",
    system_message="A human admin who provides stock data for analysis and oversees the plotting process.",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

groupchat = GroupChat(agents=[moderator, data_analyst, quality_assessor], messages=[], max_round=12)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

chat_res = moderator.initiate_chat(manager, message="Plot a chart of NVDA and TESLA stock price change YTD.", clear_history=False, config_list=llm_config["config_list"])

Moderator (to chat_manager):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
[autogen.oai.client: 08-06 10:39:15] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 08-06 10:39:17] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 08-06 10:39:18] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: DataAnalyst

[autogen.oai.client: 08-06 10:39:25] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your confi

## Example 2: Sentiment Analysis
For a sentiment analysis task, update the initiation message:

In [14]:
analyzer = AssistantAgent(
    name="Analyzer",
    system_message="You analyze the sentiment of the provided text and provide a detailed sentiment analysis.",
    llm_config=llm_config,
)

validator = AssistantAgent(
    name="Validator",
    system_message="You verify the sentiment analysis provided by the analyzer for accuracy and comprehensiveness.",
    llm_config=llm_config,
)

moderator = UserProxyAgent(
    name="Moderator",
    system_message="A human admin who provides text for analysis and oversees the sentiment analysis process.",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

groupchat = GroupChat(agents=[moderator, analyzer, validator], messages=[], max_round=12)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

chat_res = moderator.initiate_chat(manager, message="Perform sentiment analysis on the following text: 'The product was amazing and exceeded my expectations.'", clear_history=False, config_list=llm_config["config_list"])

Moderator (to chat_manager):

Perform sentiment analysis on the following text: 'The product was amazing and exceeded my expectations.'

--------------------------------------------------------------------------------
[autogen.oai.client: 08-06 10:37:11] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: Analyzer

[autogen.oai.client: 08-06 10:37:13] {329} WARNING - Model vicuna-7b-v1.5 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Analyzer (to chat_manager):

Sentiment Analysis:

The given text expresses a positive sentiment. The word "amazing" in the first sentence conveys excitement and satisfaction, while the phrase "and exceeded my expectations" indicates that the product performed better than what the user was anticipatin